# DDPM Training on CelebA Faces

This notebook demonstrates training a Denoising Diffusion Probabilistic Model on the CelebA celebrity faces dataset.

CelebA features:
- High-quality celebrity face images
- RGB color (3 channels)
- Typically resized to 64x64 or 128x128
- More challenging than MNIST, produces realistic faces

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

from realistica import (
    NoiseScheduler,
    UNet,
    DDPMTrainer,
    sample_ddpm,
    sample_ddpm_with_trajectory,
    visualize_diffusion_forward,
    EMA,
    count_parameters
)

## Configuration

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Check for MPS (Apple Silicon)
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon MPS acceleration")

# Hyperparameters
config = {
    'image_size': 64,  # CelebA images will be resized to 64x64
    'batch_size': 64,  # Adjust based on your GPU memory
    'num_epochs': 100,
    'learning_rate': 2e-4,
    'num_timesteps': 1000,
    'beta_start': 0.0001,
    'beta_end': 0.02,
    'schedule_type': 'linear',  # 'linear' or 'cosine'
    'save_interval': 10,
    'sample_interval': 10,
    'use_ema': True,  # Use exponential moving average for better samples
    'ema_decay': 0.9999,
}

# Create output directories
os.makedirs('outputs/celeba/samples', exist_ok=True)
os.makedirs('outputs/celeba/checkpoints', exist_ok=True)

## Load CelebA Dataset

The CelebA dataset will be automatically downloaded if not present.
Note: The download is ~1.4GB and may take some time.

In [ ]:
# Data preprocessing for CelebA
transform = transforms.Compose([
    transforms.Resize(config['image_size']),
    transforms.CenterCrop(config['image_size']),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Load CelebA dataset
# Note: First download may take a while
train_dataset = datasets.CelebA(
    root='./data',
    split='train',
    download=True,
    transform=transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    num_workers=4,
    pin_memory=True if device.type == 'cuda' else False,
    drop_last=True  # For stable batch size
)

print(f"Training samples: {len(train_dataset)}")
print(f"Batches per epoch: {len(train_loader)}")

## Visualize Training Images

In [ ]:
# Show some examples
sample_batch, _ = next(iter(train_loader))
grid = make_grid(sample_batch[:64], nrow=8, normalize=True, value_range=(-1, 1))
plt.figure(figsize=(12, 12))
plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
plt.axis('off')
plt.title('Sample CelebA Training Images')
plt.tight_layout()
plt.show()

## Initialize Model and Noise Scheduler

In [ ]:
# Initialize noise scheduler
noise_scheduler = NoiseScheduler(
    num_timesteps=config['num_timesteps'],
    beta_start=config['beta_start'],
    beta_end=config['beta_end'],
    schedule_type=config['schedule_type'],
    device=device
)

# Initialize U-Net model (larger for color images)
model = UNet(
    in_channels=3,  # RGB
    out_channels=3,
    base_channels=128,
    channel_multipliers=(1, 2, 3, 4),
    num_res_blocks=2,
    time_emb_dim=512,
    attention_levels=(False, False, True, True),
    dropout=0.1
).to(device)

# Count parameters
num_params = count_parameters(model)
print(f"Model parameters: {num_params:,}")
print(f"Model size: ~{num_params * 4 / 1024**2:.2f} MB (FP32)")

## Visualize the Forward Diffusion Process

In [ ]:
# Show how noise is added over time
sample_image = sample_batch[0:1].to(device)
timesteps_to_show = [0, 50, 100, 250, 500, 750, 999]

noisy_images = visualize_diffusion_forward(
    noise_scheduler,
    sample_image,
    timesteps_to_show,
    device=device
)

fig, axes = plt.subplots(1, len(timesteps_to_show), figsize=(16, 3))
for idx, (img, t) in enumerate(zip(noisy_images, timesteps_to_show)):
    axes[idx].imshow(img[0].permute(1, 2, 0).numpy() * 0.5 + 0.5)
    axes[idx].set_title(f't={t}')
    axes[idx].axis('off')

plt.suptitle('Forward Diffusion Process on CelebA', fontsize=14)
plt.tight_layout()
plt.show()

## Setup Training with DDPMTrainer

In [ ]:
# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])

# Optional: Learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=config['num_epochs'],
    eta_min=1e-6
)

# Initialize trainer
trainer = DDPMTrainer(
    model=model,
    noise_scheduler=noise_scheduler,
    optimizer=optimizer,
    device=device
)

# Optional: EMA for better sample quality
ema = None
if config['use_ema']:
    ema = EMA(model, decay=config['ema_decay'])
    print("Using EMA for model parameters")

## Define Sampling Callback

In [ ]:
def sample_and_save(trainer):
    """
    Generate and save samples during training
    """
    print("\nGenerating samples...")
    
    # Use EMA model if available
    if ema is not None:
        ema.apply_shadow()
    
    # Generate samples
    samples = sample_ddpm(
        model=trainer.model,
        noise_scheduler=trainer.noise_scheduler,
        num_samples=64,
        image_size=config['image_size'],
        channels=3,
        device=device,
        show_progress=True
    )
    
    # Restore original model if using EMA
    if ema is not None:
        ema.restore()
    
    # Save grid of samples
    grid = make_grid(samples, nrow=8, normalize=True, value_range=(-1, 1))
    save_path = f'outputs/celeba/samples/epoch_{trainer.epoch:04d}.png'
    save_image(grid, save_path)
    
    # Visualize
    plt.figure(figsize=(12, 12))
    plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
    plt.axis('off')
    plt.title(f'Generated Faces at Epoch {trainer.epoch}')
    plt.tight_layout()
    plt.show()
    
    print(f"Samples saved to {save_path}")

## Training Loop

This will take significant time depending on your hardware.
- On GPU: ~2-4 hours for 100 epochs
- On CPU: Much longer (not recommended)

You can reduce `num_epochs` for faster experimentation.

In [ ]:
# Training loop
metrics = trainer.train(
    dataloader=train_loader,
    num_epochs=config['num_epochs'],
    save_dir='outputs/celeba/checkpoints',
    save_interval=config['save_interval'],
    sample_fn=sample_and_save,
    sample_interval=config['sample_interval'],
    scheduler=lr_scheduler
)

print("\nTraining completed!")

## Plot Training Metrics

In [ ]:
# Plot epoch losses
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Epoch losses
ax1.plot(metrics['epoch_losses'], linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss per Epoch')
ax1.grid(True, alpha=0.3)

# Step losses (smoothed)
step_losses = metrics['step_losses']
ax2.plot(step_losses, alpha=0.3, label='Raw')
smoothed = np.convolve(step_losses, np.ones(100)/100, mode='valid')
ax2.plot(smoothed, linewidth=2, label='Smoothed (100-step MA)')
ax2.set_xlabel('Training Steps')
ax2.set_ylabel('Loss')
ax2.set_title('Training Loss per Step')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/celeba/training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

## Generate Final High-Quality Samples

In [ ]:
# Use EMA for best quality
if ema is not None:
    ema.apply_shadow()
    print("Using EMA model for final samples")

# Generate a large batch of samples
num_final_samples = 64
final_samples = sample_ddpm(
    model=model,
    noise_scheduler=noise_scheduler,
    num_samples=num_final_samples,
    image_size=config['image_size'],
    channels=3,
    device=device,
    show_progress=True
)

# Restore if using EMA
if ema is not None:
    ema.restore()

# Visualize
grid = make_grid(final_samples, nrow=8, normalize=True, value_range=(-1, 1))
plt.figure(figsize=(14, 14))
plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
plt.axis('off')
plt.title('Final Generated Celebrity Faces', fontsize=16)
plt.tight_layout()
plt.savefig('outputs/celeba/samples/final_samples.png', dpi=200, bbox_inches='tight')
plt.show()

## Visualize Reverse Diffusion Process

In [ ]:
# Use EMA if available
if ema is not None:
    ema.apply_shadow()

# Generate with intermediate steps
intermediate_steps = sample_ddpm_with_trajectory(
    model=model,
    noise_scheduler=noise_scheduler,
    num_samples=8,
    image_size=config['image_size'],
    channels=3,
    device=device,
    num_steps_to_save=10
)

if ema is not None:
    ema.restore()

# Visualize the reverse process
fig, axes = plt.subplots(8, len(intermediate_steps), figsize=(16, 12))
for sample_idx in range(8):
    for step_idx, step_images in enumerate(intermediate_steps):
        img = step_images[sample_idx].permute(1, 2, 0).numpy() * 0.5 + 0.5
        img = np.clip(img, 0, 1)
        axes[sample_idx, step_idx].imshow(img)
        axes[sample_idx, step_idx].axis('off')
        if sample_idx == 0:
            axes[sample_idx, step_idx].set_title(f'Step {step_idx}', fontsize=10)

plt.suptitle('Reverse Diffusion: From Noise to Celebrity Face', fontsize=16)
plt.tight_layout()
plt.savefig('outputs/celeba/samples/reverse_diffusion.png', dpi=150, bbox_inches='tight')
plt.show()

## Interpolation in Latent Space

In [ ]:
# Generate two random starting noises and interpolate
if ema is not None:
    ema.apply_shadow()

# Two random noise vectors
noise1 = torch.randn(1, 3, config['image_size'], config['image_size'], device=device)
noise2 = torch.randn(1, 3, config['image_size'], config['image_size'], device=device)

# Create interpolation
num_interpolations = 8
alphas = torch.linspace(0, 1, num_interpolations).to(device)

interpolated_samples = []
for alpha in tqdm(alphas, desc='Interpolating'):
    # Interpolate noise
    interpolated_noise = (1 - alpha) * noise1 + alpha * noise2
    
    # Denoise
    x = interpolated_noise.clone()
    for t in reversed(range(noise_scheduler.num_timesteps)):
        t_batch = torch.tensor([t], device=device)
        predicted_noise = model(x, t_batch)
        
        alpha_t = noise_scheduler.alphas[t]
        alpha_cumprod = noise_scheduler.alphas_cumprod[t]
        beta = noise_scheduler.betas[t]
        
        if t > 0:
            noise = torch.randn_like(x)
        else:
            noise = torch.zeros_like(x)
        
        x = (
            1 / torch.sqrt(alpha_t) * (
                x - (beta / torch.sqrt(1 - alpha_cumprod)) * predicted_noise
            ) + torch.sqrt(beta) * noise
        )
    
    interpolated_samples.append(x)

if ema is not None:
    ema.restore()

# Visualize interpolation
interpolated_grid = torch.cat(interpolated_samples, dim=0)
grid = make_grid(interpolated_grid, nrow=num_interpolations, normalize=True, value_range=(-1, 1))

plt.figure(figsize=(16, 3))
plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
plt.axis('off')
plt.title('Latent Space Interpolation', fontsize=14)
plt.tight_layout()
plt.savefig('outputs/celeba/samples/interpolation.png', dpi=150, bbox_inches='tight')
plt.show()

## Save Final Model with EMA

In [ ]:
# Save the final trained model
save_dict = {
    'model_state_dict': model.state_dict(),
    'config': config,
}

if ema is not None:
    ema.apply_shadow()
    save_dict['ema_state_dict'] = {k: v.cpu() for k, v in model.state_dict().items()}
    ema.restore()

torch.save(save_dict, 'outputs/celeba/checkpoints/final_model_with_ema.pt')
print("Final model (with EMA) saved!")

## Load and Use Saved Model

In [ ]:
# Example: How to load and use the saved model
checkpoint = torch.load('outputs/celeba/checkpoints/final_model_with_ema.pt', map_location=device)

# Load EMA weights if available
if 'ema_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['ema_state_dict'])
    print("Loaded EMA model weights")
else:
    model.load_state_dict(checkpoint['model_state_dict'])
    print("Loaded regular model weights")

# Now you can generate samples with the loaded model
model.eval()
print("Model loaded and ready for inference!")